### *Вопросно ответная система типа "retriever-reader"*

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

import re
import os
# Сохранение кеша по дефолтному пути невозможно из-за нехватки места
cache_dir = '/usr/src/app/!datasets/'
os.environ["TRANSFORMERS_CACHE"] = cache_dir

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
class QASystem:
    def __init__(self, retriever='DeepPavlov/rubert-base-cased', reader='AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru', cache_dir='~/.cache/huggingface/datasets'):
        # Загрузка претренированных моделей и токенайзеров ретривера и ридера
        self.tokenizer_retriever = BertTokenizer.from_pretrained(retriever, do_lower_case=True, cache_dir=cache_dir)
        self.model_retriever = BertModel.from_pretrained(retriever, output_hidden_states = True, cache_dir=cache_dir)
        self.model_retriever.to(device)
        self.model_retriever.eval()

        self.tokenizer_reader = AutoTokenizer.from_pretrained(reader, cache_dir=cache_dir)
        self.model_reader = AutoModelForQuestionAnswering.from_pretrained(reader, cache_dir=cache_dir)
        self.model_reader.to(device)
        self.model_reader.eval()

    # Сохранение текстов и преобразование в векторы
    def load_text(self, texts):
        self.texts = texts
        self.texts_vectors = [self._text2vec(text).numpy() for text in texts]
    
    # Преобразовать текст в вектор с помощью токенайзера ретривера
    def _text2vec(self, text):
        inputs = self.tokenizer_retriever(text, return_tensors="pt", truncation=True, max_length=512)
        inputs = {name: tensor.to(device) for name, tensor in inputs.items()}
        outputs = self.model_retriever(**inputs)
        return outputs.last_hidden_state.mean(dim=1).detach().cpu()

    def retriever(self, question):
        # Преобразование вопроса в вектор
        question_vector = self._text2vec(question).numpy()
        # Измерение сходства по косинусной мере между векторами
        similarities = [cosine_similarity(text_vector, question_vector) for text_vector in self.texts_vectors]
        # Берем наиболее подходящий текст
        max_similarity_index = similarities.index(max(similarities))
        return self.texts[max_similarity_index], max_similarity_index

    def reader(self, question, text):
        # Преобразовать текст в вектор с помощью токенайзера ретривера
        encoding = self.tokenizer_reader.encode_plus(text=question,text_pair=text)
        inputs = encoding['input_ids']
        tokens = self.tokenizer_reader.convert_ids_to_tokens(inputs)
        inputs = torch.tensor([inputs]).to(device)
        output = self.model_reader(input_ids=inputs)

        start_scores = output[0][0]
        end_scores = output[1][0]

        # Поиск начала и конца ответа с наибольшим суммарным скором
        max_score = float('-inf')
        start_index = end_index = 0

        for i in range(len(start_scores)):
            for j in range(i, len(end_scores)):
                score = start_scores[i] + end_scores[j]
                if score > max_score:
                    max_score = score
                    start_index = i
                    end_index = j
                    
        # Преобразование токенов ответа в строку
        answer_tokens = tokens[start_index:end_index+1]
        answer = self.tokenizer_reader.convert_tokens_to_string(answer_tokens)
        return answer

    def _preprocess_text(self, text):
        # Удаляем все знаки препинания в конце строки
        text = re.sub(r'[^\w\s]$', '', text)
        # Добавляем точку в конец строки
        text = text.strip() + '.'
        # Преобразуем первую букву строки в заглавную
        text = text[0].upper() + text[1:]
        return text

    def __call__(self, question):
        retrieved_text, text_index = self.retriever(question)
        answer = self.reader(retrieved_text, question)
        answer = self._preprocess_text(answer)
        return answer, text_index


In [4]:
# Загрузка датасета sberquad
from datasets import load_dataset
dataset = load_dataset("sberquad", cache_dir=cache_dir)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 45328
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5036
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 23936
    })
})

In [5]:
# Отбор уникальных текстов
def get_unique_texts(texts):
    return list(set(texts))

texts = [item['context'] for item in dataset['test']]#[10:1050:50]
texts = get_unique_texts(texts)
print(len(texts))

4425


### Демонстрация работы

In [6]:
qa_system = QASystem(cache_dir=cache_dir)
# Загрузка текстов для предварительной обработки 
qa_system.load_text(texts)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
questions = [
    "Какой переход характерен дифтонгам?",
    "Что такое гармонические колебания?",
    "Как гасятся кредитные линии по кредитным картам?",
    "Какие породы собак официально признаны опасными для человека?",
    "Что произошло в эпизоде Приключение суссекского вампира истории о Шерлоке Холмсе?",
    "Какой банк входит в группу наиболее ликвидных банков РФ?",
    "Какие фильмы для RKO Pictures выпустил Вэл Льютон?",
    "Чем отличается многоклеточность и колониальность?",
    "С какого места исполняется свободный бросок?"
            ]

idxs = [] # индексы текстов, из которых брался ответ
for i, question in enumerate(questions):
    answer, idx = qa_system(question)
    idxs.append(idx)
    print(f"{i}) {question}\nОтвет: {answer} [текст №{idx}]")


0) Какой переход характерен дифтонгам?
Ответ: От одного гласного звукотипа к другому. [текст №4155]
1) Что такое гармонические колебания?
Ответ: Орбитальные элементы. [текст №145]
2) Как гасятся кредитные линии по кредитным картам?
Ответ: Надлежащим оформлением дел по межбанковским кредитам. Предоставление межбанковских кредитов должно сопровождаться открытием счетов. [текст №3948]
3) Какие породы собак официально признаны опасными для человека?
Ответ: Питбультерьеров, тоса-ину или японских мастифов, аргентинских догов и фила бразильеро. [текст №1081]
4) Что произошло в эпизоде Приключение суссекского вампира истории о Шерлоке Холмсе?
Ответ: Артур Конан Дойль упомянул, что доктор Ватсон в прошлом выступал за регбийную команду Блекхит. [текст №3656]
5) Какой банк входит в группу наиболее ликвидных банков РФ?
Ответ: ЧЕЛИНДБАНК. [текст №267]
6) Какие фильмы для RKO Pictures выпустил Вэл Льютон?
Ответ: Люди-кошки (1942), Я гуляла с зомби (1943), и Похитители тел (1945). [текст №3904]
7) Че

#### Правильные ответы:
1\. Колебания, при которых физическая величина изменяется с течением времени по гармоническому (синусоидальному, косинусоидальному) закону. \
5\. ПТБ. \
7\. У колониальных организмов отсутствуют настоящие дифференцированные клетки, а следовательно, и разделение тела на ткани.

In [8]:
# Тексты, в которых был найден ответ
for i in idxs:
    print(f"{i}) {texts[i]}")

4155) До сих пор речь шла лишь о монофтонгах, то есть таких гласных, акустическая картина которых не меняется на всем протяжении их артикуляции. Под дифтонгами понимаются звуки, артикуляция которых подразумевает переход от одного гласного звукотипа к другому; трифтонги включают, соответственно, три звукотипа. Обычно в составе дифтонгов один из компонентов является слоговым, а прочие — нет. Если слоговым является первый компонент, то такой дифтонг называется нисходящим, если второй — восходящим. В роли неслоговых компонентов чаще всего выступают неслоговые соответствия закрытым гласным, то есть [w] и [j], например в английском [kaɪt] воздушный змей , loʊ низкий ; однако встречаются и другие варианты, например в древнеанглийском языке были нисходящие дифтонги [æa] и [æo]. Очень редко встречаются равновесные дифтонги, например, в нивхском языке.
145) Определение орбит небесных тел является одной из задач небесной механики. Для задания орбиты спутника планеты, астероида или Земли использую